# Scraping IGN

In [2]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from bs4 import BeautifulSoup
import time
options = Options()
options.headless = True
browser = webdriver.Firefox(options=options)

def loadAndScroll(browser, url, scrolls, t):
    browser.get(url)

    for i in range(scrolls):
        browser.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        time.sleep(t)
    
    doc = BeautifulSoup(browser.page_source, 'html.parser')
    return doc
        

In [29]:
url = "https://nordic.ign.com/article/review?keyword__type=game"
doc = loadAndScroll(browser, url, 20,1)
headlines = doc.find_all('h3')

In [30]:

with open('linksIGN.txt','w') as f:
    for headline in headlines:
        link = headline.find('a')
        if link:
            f.write(str(link.get('href'))+ '\n')
            #print(link.get('href'))
            

In [ ]:
import re
import csv

pattern = re.compile(r'<p id="id_keyword__.*?__text">')
rows = [['Game', 'Score', 'Text']]
with open('linksIGN.txt','r') as f:
    i = 0
    for url in f:
        doc = loadAndScroll(browser,url, 0, 0.0)
        score = doc.find_all('span', class_="side-wrapper side-wrapper hexagon-content")
        try:
            #ugly hack but works
            score = str(score).split('div')[1]
            if score[2] == '<':
                score = score[1]
            else:
                score = score[1:3] #10
            
        except:
            continue
            score = "-1"
        title = str(doc.find_all('title')[0])
        title = title.strip('<title>').strip('Review</title>')
        
        text = doc.find_all('p')
        review_text = ''
        for section in text:
            t = str(section)
            review_text += t + ' '
        
        rows.append([title, score, review_text])
        
        i += 1
        print(i)
        
with open('IgnReviews.csv', 'w') as csvfile:
        csvwriter = csv.writer(csvfile)  
        csvwriter.writerows(rows)

In [85]:
with open('IgnReviews2.csv', 'w') as csvfile:
        csvwriter = csv.writer(csvfile)  
        csvwriter.writerows(rows)

In [94]:
import pandas as pd
df = pd.read_csv('IgnReviews.csv')
pattern = re.compile(r'<(.*?)>')

for entry in df.itertuples():
    txt = entry.Text
    new = txt
    matches =  re.finditer(pattern, str(txt))
    for m in matches:
        try:
            new = re.sub(m.group(),'',new)
        except:
            print(entry.Index, entry.Game, m)
    
    df.loc[entry.Index,'Text'] = new

df.to_csv('IgnClean2.csv')
    

# Scrape Gamestop

In [ ]:
options = Options()
options.headless = True
browser = webdriver.Firefox(options=options)

baseUrl = "https://www.gamespot.com/games/reviews/?page="
with open('linksGamespot.txt','w') as f:
    for i in range(1,201):
        url = baseUrl + str(i)
        doc = loadAndScroll(browser, url, 1,0.5)
        links = doc.find_all('a')
        for link in links:
            class_ = link.get('class')
            if class_ == ['card-item__link', 'text-decoration--none']:
                href = link.get('href')
                f.write("https://www.gamespot.com" + href + '\n')

In [5]:
import re

def getScoreGS(divs):
    score = "-1"
    for d in divs:
        class_ = d.get('class')
        if class_ == ['review-ring-score__score', 'text-bold']:
            score = str(d).strip("</div>")
            idx = score.find('>') + 1
            score = score[idx:]
    return score
            
def getTextGS_og(texts):
    txt = ""
    for p in texts:
        print(p)
        if p.get('dir') == "ltr":
            t = str(p)
            
            t = t.strip('<p dir=\"ltr\">').strip("</p>")
            txt += t
    return txt

def getTextGS(divs):
    txt = ""
    for d in divs:
        class_ = d.get('class')
        if class_ == ['js-content-entity-body']:
            ps = d.find_all('p')
            ps = [str(x) for x in ps]
            indices = []
            for i,p in enumerate(ps):
                if "By clicking 'enter', you agree to GameSpot's" in p:
                    indices.append(i)
                if "Want us to remember this setting for all your devices?" in p:
                    indices.append(i)
                if "class=\"video-caption\"" in p:
                    indices.append(i)
                if "button class=\"js-vid-play" in p:
                    indices.append(i)
                if "class=\"av-video-player-no-js" in p:
                    indices.append(i)
                if "Sign in</a> now!" in p:
                    indices.append(i)
                
            strs = [v for i, v in enumerate(ps) if i not in indices]
                    
            txt = ' '.join(strs)
            #print(txt)
       
    return txt


def getGameGS(lists):
    pattern = re.compile(r'<span>(.*?)</span>')
    game = ""
    for li in lists:
        #print(li.get('class') )
        if li.get('class') == ['subnav-list__item', 'subnav-list__item-primary']:
            string = str(li)
            match = re.search(pattern, string)
            if match:
                game = match.group(1).strip()
    return game


In [84]:
#Clear all double entries

f =  open('linksGamespot.txt','r')
rows = f.readlines()

last = []
with open('linksGSClean.txt','w') as f:
    for link in rows:
        link_str = str(link)
        current = link_str.split('/')
        current = current[:-2]
        
        if last == current:
            #print(current[-1], last[-1])
            continue
        last = current
        f.write(link)

In [113]:
import csv

"""row = ['Game', 'Score', 'Text']
with open('GamespotReviewTest.csv', 'a') as csvfile:
        csvwriter = csv.writer(csvfile)  
        csvwriter.writerow(row)"""
        

with open('linksGSClean.txt','r') as f:
    urls =  f.readlines()
tot = len(urls)
with open('GamespotReviewTest.csv', 'a') as csvfile:
    csvwriter = csv.writer(csvfile)  
    for i, url in enumerate(urls):
        #temp fix
        if i < 2910:
            continue
        
        doc = loadAndScroll(browser,url, 1, 2)
        divs = doc.find_all('div')
        texts = doc.find_all('p')
        lists = doc.find_all('li')
        game = getGameGS(lists)
        score = getScoreGS(divs)
        text = getTextGS(divs)
        row = [game, score, text]
        csvwriter.writerow(row)
            
        print(str(i) + '/' + str(tot))



2910/3840
2911/3840
2912/3840
2913/3840
2914/3840
2915/3840
2916/3840
2917/3840
2918/3840
2919/3840
2920/3840
2921/3840
2922/3840
2923/3840
2924/3840
2925/3840
2926/3840
2927/3840
2928/3840
2929/3840
2930/3840
2931/3840
2932/3840
2933/3840
2934/3840
2935/3840
2936/3840
2937/3840
2938/3840
2939/3840
2940/3840
2941/3840
2942/3840
2943/3840
2944/3840
2945/3840
2946/3840
2947/3840
2948/3840
2949/3840
2950/3840
2951/3840
2952/3840
2953/3840
2954/3840
2955/3840
2956/3840
2957/3840
2958/3840
2959/3840
2960/3840
2961/3840
2962/3840
2963/3840
2964/3840
2965/3840
2966/3840
2967/3840
2968/3840
2969/3840
2970/3840
2971/3840
2972/3840
2973/3840
2974/3840
2975/3840
2976/3840
2977/3840
2978/3840
2979/3840
2980/3840
2981/3840
2982/3840
2983/3840
2984/3840
2985/3840
2986/3840
2987/3840
2988/3840
2989/3840
2990/3840
2991/3840
2992/3840
2993/3840
2994/3840
2995/3840
2996/3840
2997/3840
2998/3840
2999/3840
3000/3840
3001/3840
3002/3840
3003/3840
3004/3840
3005/3840
3006/3840
3007/3840
3008/3840
3009/3840


In [9]:
def ten2tree(val):
    if val < 5:
        return "0"
    elif val > 6:
        return "2"
    return "1"
    
def ten2five(val):
    if val < 3:
        return "1"
    elif val < 5:
        return "2"
    elif val > 8:
        return "5"
    elif val > 6:
        return "4"
    
    return "3"
    

In [4]:
import pandas as pd
import re
df = pd.read_csv('GSClean3.csv')
pattern = re.compile(r'a href=(.*)>')
drop_idx = []
for entry in df.itertuples():
    
    txt = entry.Text
    #remove weird entries
    if len(str(txt)) < 100:
        drop_idx.append(entry.Index)
        print(entry)
        continue
    score = round(entry.Score)
    
    new = txt
    matches =  re.finditer(pattern, str(txt))
    for m in matches:
        try:
            new = re.sub(m.group(),'',new)
        except:
            print(entry.Index, entry.Game, m)
        print(entry.Index, entry.Game, m)
    
    df.loc[entry.Index,'Text'] = new
    df.loc[entry.Index,'Score'] = score
    df.loc[entry.Index,'3-pt'] = ten2tree(score)
    df.loc[entry.Index,'5-pt'] = ten2five(score)
for i in reversed(drop_idx):
    df = df.drop([i])
header = ['Game','Score','3-pt','5-pt','Text']
df.to_csv('GSClean3.csv', columns=header, index=False)

C:\Users\tomas\AppData\Local\Temp\ipykernel_16396\1599866436.py:27: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[entry.Index,'3-pt'] = ten2tree(score)
C:\Users\tomas\AppData\Local\Temp\ipykernel_16396\1599866436.py:28: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '4' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[entry.Index,'5-pt'] = ten2five(score)


# PC GAMER

In [113]:
urlArchive = "https://www.pcgamer.com/reviews/archive/"

options = Options()
options.headless = True
browser = webdriver.Firefox(options=options)



In [114]:
doc = loadAndScroll(browser, urlArchive, 1,0.5)
li = doc.find_all('ul')
with open('linksPCGarchive.txt','w') as f:
    for l in li:
        as_ = l.find_all('a')
        for a in as_:
            link = a['href']
            link_str = str(link)
            if link_str.startswith("https://www.pcgamer.com/reviews/archive/"):
                f.write(link_str + '\n')

In [ ]:
f =  open('linksPCGarchive.txt','r')
rows = f.readlines()

with open('linksPCG.txt','w') as f:
    for link in rows:
        doc = loadAndScroll(browser, link, 1,0.5)
        lists = doc.find_all('li')
        for li in lists:
            class_ = li.get('class')
            if class_ == ['day-article']:
                a = li.find_all('a')
                f.write(a[0]['href'] + '\n')


Remove entries containing specific word

In [231]:
f =  open('linksPCGClean.txt','r')
rows = f.readlines()

with open('linksPCGClean.txt','w') as f:
    for link in rows:
        if "nas-" in link:
            print(link)
            continue
        f.write(link)
        #1034

https://www.pcgamer.com/tiny-tinas-wonderlands-review/

https://www.pcgamer.com/thecus-n5810pro-nas-review/

https://www.pcgamer.com/qnap-tvs-871-nas-review/

https://www.pcgamer.com/qnap-turbonas-ts-451-review/

https://www.pcgamer.com/qnap-tvs-ec1080-10-bay-nas-review/



In [8]:
test = "https://www.gamespot.com/reviews/call-of-duty-4-modern-warfare-review/1900-6182426/"

doc =loadAndScroll(browser,test,1,1)
texts = doc.find_all('p')
text = getTextGS(texts)
print(text)

# Gamestop < 6


In [4]:
baseUrl = 'https://www.gamespot.com/games/reviews/?review_filter_type[platform]=&review_filter_type[genre]=&review_filter_type[timeFrame]=4&review_filter_type[startDate]=01/01/2000&review_filter_type[endDate]=01/01/2010&review_filter_type[minRating]=&review_filter_type[theme]=&review_filter_type[region]=&review_filter_type[letter]=&sort=gs_score_desc&page='

options = Options()
options.headless = True
browser = webdriver.Firefox(options=options)

with open('linksGamespotExtra.txt','w') as f:
    for i in range(325,396):
        url = baseUrl + str(i)
        doc = loadAndScroll(browser, url, 1,0.5)
        links = doc.find_all('a')
        for link in links:
            class_ = link.get('class')
            if class_ == ['card-item__link', 'text-decoration--none']:
                href = link.get('href')
                f.write("https://www.gamespot.com" + href + '\n')

In [6]:
f =  open('linksGamespotExtra.txt','r')
rows = f.readlines()

last = []
with open('linksGSExtraClean.txt','w') as f:
    for link in rows:
        link_str = str(link)
        current = link_str.split('/')
        current = current[:-2]
        
        if last == current:
            #print(current[-1], last[-1])
            continue
        last = current
        f.write(link)

In [8]:
import csv

with open('linksGSExtraClean.txt','r') as f:
    urls =  f.readlines()
tot = len(urls)
with open('GamespotExtraReviewTest.csv', 'a') as csvfile:
    csvwriter = csv.writer(csvfile)  
    for i, url in enumerate(urls):
        
        doc = loadAndScroll(browser,url, 1, 2)
        divs = doc.find_all('div')
        texts = doc.find_all('p')
        lists = doc.find_all('li')
        game = getGameGS(lists)
        score = getScoreGS(divs)
        text = getTextGS(divs)
        row = [game, score, text]
        csvwriter.writerow(row)
            
        print(str(i) + '/' + str(tot))

0/1421
1/1421
2/1421
3/1421
4/1421
5/1421
6/1421
7/1421
8/1421
9/1421
10/1421
11/1421
12/1421
13/1421
14/1421
15/1421
16/1421
17/1421
18/1421
19/1421
20/1421
21/1421
22/1421
23/1421
24/1421
25/1421
26/1421
27/1421
28/1421
29/1421
30/1421
31/1421
32/1421
33/1421
34/1421
35/1421
36/1421
37/1421
38/1421
39/1421
40/1421
41/1421
42/1421
43/1421
44/1421
45/1421
46/1421
47/1421
48/1421
49/1421
50/1421
51/1421
52/1421
53/1421
54/1421
55/1421
56/1421
57/1421
58/1421
59/1421
60/1421
61/1421
62/1421
63/1421
64/1421
65/1421
66/1421
67/1421
68/1421
69/1421
70/1421
71/1421
72/1421
73/1421
74/1421
75/1421
76/1421
77/1421
78/1421
79/1421
80/1421
81/1421
82/1421
83/1421
84/1421
85/1421
86/1421
87/1421
88/1421
89/1421
90/1421
91/1421
92/1421
93/1421
94/1421
95/1421
96/1421
97/1421
98/1421
99/1421
100/1421
101/1421
102/1421
103/1421
104/1421
105/1421
106/1421
107/1421
108/1421
109/1421
110/1421
111/1421
112/1421
113/1421
114/1421
115/1421
116/1421
117/1421
118/1421
119/1421
120/1421
121/1421
122/1421
123

In [21]:
import pandas as pd
import re
df = pd.read_csv('GamespotExtraReviewTest.csv')
# a href=(.*)>
pattern = re.compile(r'<(.*?)>')
drop_idx = []
for entry in df.itertuples():
    txt = entry.Text
    #remove weird entries
    if len(str(txt).split()) < 100:
        drop_idx.append(entry.Index)
        print(entry)
        continue
    score = round(entry.Score)
    
    new = txt
    matches =  re.finditer(pattern, str(txt))
    for m in matches:
        try:
            new = re.sub(m.group(),'',new)
        except:
            print(entry.Index, entry.Game, m)
        print(entry.Index, entry.Game, m)
    
    df.loc[entry.Index,'Text'] = new
    df.loc[entry.Index,'Score'] = score
    df.loc[entry.Index,'3-pt'] = ten2tree(score)
    df.loc[entry.Index,'5-pt'] = ten2five(score)
for i in reversed(drop_idx):
    df = df.drop([i])
header = ['Game','Score','3-pt','5-pt','Text']
df.to_csv('GSExtraClean.csv', columns=header, index=False)

0 Legends of Wrestling <re.Match object; span=(0, 3), match='<p>'>
0 Legends of Wrestling <re.Match object; span=(653, 657), match='</p>'>
0 Legends of Wrestling <re.Match object; span=(658, 661), match='<p>'>
0 Legends of Wrestling <re.Match object; span=(1328, 1332), match='</p>'>
0 Legends of Wrestling <re.Match object; span=(1333, 1336), match='<p>'>
0 Legends of Wrestling <re.Match object; span=(2562, 2566), match='</p>'>
0 Legends of Wrestling <re.Match object; span=(2567, 2570), match='<p>'>
0 Legends of Wrestling <re.Match object; span=(3456, 3460), match='</p>'>
0 Legends of Wrestling <re.Match object; span=(3461, 3464), match='<p>'>
0 Legends of Wrestling <re.Match object; span=(4638, 4641), match='<i>'>
0 Legends of Wrestling <re.Match object; span=(4647, 4651), match='</i>'>
0 Legends of Wrestling <re.Match object; span=(4945, 4949), match='</p>'>
0 Legends of Wrestling <re.Match object; span=(4950, 4953), match='<p>'>
0 Legends of Wrestling <re.Match object; span=(4953, 49

C:\Users\tomas\AppData\Local\Temp\ipykernel_13784\2116528351.py:27: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '1' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[entry.Index,'3-pt'] = ten2tree(score)
C:\Users\tomas\AppData\Local\Temp\ipykernel_13784\2116528351.py:28: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '3' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[entry.Index,'5-pt'] = ten2five(score)


319 Delta Force: Xtreme 2 <re.Match object; span=(0, 3), match='<p>'>
319 Delta Force: Xtreme 2 <re.Match object; span=(667, 671), match='</p>'>
319 Delta Force: Xtreme 2 <re.Match object; span=(672, 675), match='<p>'>
319 Delta Force: Xtreme 2 <re.Match object; span=(676, 680), match='</p>'>
319 Delta Force: Xtreme 2 <re.Match object; span=(681, 684), match='<p>'>
319 Delta Force: Xtreme 2 <re.Match object; span=(684, 688), match='</p>'>
319 Delta Force: Xtreme 2 <re.Match object; span=(689, 692), match='<p>'>
319 Delta Force: Xtreme 2 <re.Match object; span=(1270, 1274), match='</p>'>
319 Delta Force: Xtreme 2 <re.Match object; span=(1275, 1278), match='<p>'>
319 Delta Force: Xtreme 2 <re.Match object; span=(2372, 2376), match='</p>'>
319 Delta Force: Xtreme 2 <re.Match object; span=(2377, 2380), match='<p>'>
319 Delta Force: Xtreme 2 <re.Match object; span=(3028, 3032), match='</p>'>
319 Delta Force: Xtreme 2 <re.Match object; span=(3033, 3036), match='<p>'>
319 Delta Force: Xtreme 

In [29]:
df1 = pd.read_csv('GSClean3.csv')
df2 = pd.read_csv('GSExtraClean.csv')
df3 = pd.merge(df1,df2,how= 'outer')
df3 = df3.drop('5-pt',axis=1)
df3

df3.to_csv('GSDatabase.csv', index=False)